In [1]:
!pip install -q trl transformers accelerate peft datasets bitsandbytes einops tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0

In [24]:
from datasets import load_dataset

# Path to your CSV file
csv_file_path = '/content/train.csv'

# Load the CSV file as a Dataset object
dataset = load_dataset('csv', data_files=csv_file_path)

In [2]:
from datasets import load_dataset
billsum = load_dataset("billsum", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [15]:
billsum

Dataset({
    features: ['text', 'summary', 'title', 'formated text'],
    num_rows: 18949
})

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [25]:
from datasets import DatasetDict

# Assuming `dataset_dict` is your DatasetDict object
# which has been defined somewhere in your code.

# Extract the 'train' Dataset from the DatasetDict
chat = dataset['train']

# Now `train_dataset` is a standalone Dataset object
print(chat)


Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})


In [19]:
chat[0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [26]:
def format_input(example: dict) -> str:
  example[ 'formated text'] = f''' Human:act like a psychologist and respond for the following question: {example["Context"]}
  ##Assistant: {example["Response"]}'''
  return example

In [27]:
chat = chat.map(format_input)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [8]:
billsum = billsum.map(format_input)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

In [22]:
chat

Dataset({
    features: ['Context', 'Response', 'formated text'],
    num_rows: 3512
})

In [28]:
chat[0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [31]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import prepare_model_for_kbit_training
# from peft import prepare_model_for_kbit_training # This line seems to be incorrect or from a custom module

model_name = "ybelkada/falcon-7b-sharded-bf16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map='auto'
)

model.config.use_cache = False
model.gradient_checkpointing_enable()


config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/921M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [32]:
model = prepare_model_for_kbit_training(model) # This line seems to be incorrect or from a custom module

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [34]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
  lora_alpha=32,
  lora_dropout=0.05,
  r=16,
  bias="none",
  task_type=TaskType.CAUSAL_LM,
  target_modules=["query_key_value"]
)

In [36]:
from peft import get_peft_model
model = get_peft_model (model, peft_config)

In [39]:
from transformers import TrainingArguments
training_arguments = TrainingArguments(
  output_dir='/training_ result',
  per_device_train_batch_size=1,
  gradient_accumulation_steps=4,
  num_train_epochs=1,
  learning_rate=2e-4,
  fp16=True,
  optim='paged_adamw_8bit',
  lr_scheduler_type='cosine',
  warmup_ratio=0.05
)

In [41]:
from trl import SFTTrainer

trainer = SFTTrainer(
  model=model,
  train_dataset=chat,
  peft_config=peft_config,
  dataset_text_field="formated text",
  tokenizer=tokenizer,
  args=training_arguments,
  max_seq_length=1024
)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_do

Step,Training Loss


In [ ]:
trained_model_dir = './trained_model'
model.save_pretrained (trained_model_dir)

In [ ]:
from peft import PeftConfig, PeftModel

config = PeftConfig.from_pretrained(trained_model_dir)
trained_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  return_dict=True,
  quantization_config=bnb_config,
  trust_remote_code=True,
  device_map='auto'
  )

trained_model = PeftModel.from_pretrained(trained_model, trained_model_dir)
trained_model_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
trained_model_tokenizer.pad_token = trained_model_tokenizer.eos_token

In [ ]:
generation_config = trained_model.generation_config
generation_config.max_new_token = 1024
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequence = 1
generation_config.pad_token_id = trained_model_tokenizer.pad_token_id
generation_config.eos_token_id = trained_model_tokenizer.eos_token_id

In [ ]:
device='cuda'

In [ ]:
query='''jaknfkjanf'''

In [ ]:
prompt = f'''###Human: Summerize the given text: {query}
        ### Assistant:'''

In [ ]:
encodings = trained_model_tokenizer(prompt,return_tensors='pt').to(device)

In [ ]:
encodings

In [ ]:
with torch.inference_mode():
  outputs = trained_model.generate(
  input_ids=encodings.input_ids,
  attention_mask=encodings.attention_mask,
  generation_config=generation_config,
  max_new_tokens=100
  )

In [ ]:
outputs

In [ ]:
outputs = trained_model_tokenizer.decode(outputs[0],skip_special_tokens=True)
outputs